In [1]:
import os

os.chdir('D:\\dev\\projects\\artifact-aware-assistant\\backend\\app\\routes\\api')
os.environ['no_proxy'] = "10.227.91.60"

from conversation import SYSTEM_MESSAGE, Artifact, Tool
# from example_tools import tools

from dotenv import load_dotenv

load_dotenv()  # Load environment variables

True

In [2]:
def get_listing(address):
    content = f"""\
{{
    "address": "{address}",
    "city": "Cedar Rapids",
    "state": "IA",
    "zip": "52402",
    "price": 185000,
    "beds": 3,
    "baths": 2,
    "sqft": 1450,
    "lot_size": 0.25,
    "year_built": 1978,
    "description": "Charming ranch-style home in established neighborhood. Updated kitchen with new appliances. Finished basement, attached 2-car garage, fenced backyard with mature trees. Close to schools and shopping.",
    "features": [           
        "Central air",
        "Forced air heating",
        "Hardwood floors",
        "Updated kitchen",
        "Finished basement",
        "Attached garage",
        "Fenced yard"
    ],
}}
"""

    artifact = Artifact(
        identifier="18bacG4a",
        type="application/json",
        title=address,
        content=content)
    return str(artifact)


get_listing_schema = {
    "name": "get_listing",
    "description": "Get details about a specific property listing",
    "input_schema": {
        "type": "object",
        "properties": {
            "address": {
                "type": "string",
                "description": "The street address to look up"
            }
        },
        "required": ["address"]
    }
}


tools = [
    Tool(get_listing_schema, get_listing),
]

In [3]:
get_listing('192 Oak Street')

'<artifact identifier="18bacG4a" type="application/json" title="192 Oak Street">\n{\n    "address": "192 Oak Street",\n    "city": "Cedar Rapids",\n    "state": "IA",\n    "zip": "52402",\n    "price": 185000,\n    "beds": 3,\n    "baths": 2,\n    "sqft": 1450,\n    "lot_size": 0.25,\n    "year_built": 1978,\n    "description": "Charming ranch-style home in established neighborhood. Updated kitchen with new appliances. Finished basement, attached 2-car garage, fenced backyard with mature trees. Close to schools and shopping.",\n    "features": [           \n        "Central air",\n        "Forced air heating",\n        "Hardwood floors",\n        "Updated kitchen",\n        "Finished basement",\n        "Attached garage",\n        "Fenced yard"\n    ],\n}\n\n</artifact>'

In [4]:
print(get_listing('192 Oak Street'))

<artifact identifier="18bacG4a" type="application/json" title="192 Oak Street">
{
    "address": "192 Oak Street",
    "city": "Cedar Rapids",
    "state": "IA",
    "zip": "52402",
    "price": 185000,
    "beds": 3,
    "baths": 2,
    "sqft": 1450,
    "lot_size": 0.25,
    "year_built": 1978,
    "description": "Charming ranch-style home in established neighborhood. Updated kitchen with new appliances. Finished basement, attached 2-car garage, fenced backyard with mature trees. Close to schools and shopping.",
    "features": [           
        "Central air",
        "Forced air heating",
        "Hardwood floors",
        "Updated kitchen",
        "Finished basement",
        "Attached garage",
        "Fenced yard"
    ],
}

</artifact>


In [5]:
t = Tool(get_listing_schema, get_listing)
t

In [6]:
print(t.callable(**{'address': '192 Oak St'}))

<artifact identifier="18bacG4a" type="application/json" title="192 Oak St">
{
    "address": "192 Oak St",
    "city": "Cedar Rapids",
    "state": "IA",
    "zip": "52402",
    "price": 185000,
    "beds": 3,
    "baths": 2,
    "sqft": 1450,
    "lot_size": 0.25,
    "year_built": 1978,
    "description": "Charming ranch-style home in established neighborhood. Updated kitchen with new appliances. Finished basement, attached 2-car garage, fenced backyard with mature trees. Close to schools and shopping.",
    "features": [           
        "Central air",
        "Forced air heating",
        "Hardwood floors",
        "Updated kitchen",
        "Finished basement",
        "Attached garage",
        "Fenced yard"
    ],
}

</artifact>


In [7]:
def process_tool_uses_and_results(messages):
    '''
    {'role': 'assistant', 
    'content': [
        {'text': "I'll help you get information about the property at 192 Oak St.", 'type': 'text'}, 
        {'id': 'toolu_012m2VtyXxFL4RBGHzJqSN4R', 'input': {'address': '192 Oak St'}, 'name': 'get_listing', 'type': 'tool_use'}
        ]
    }
    
    {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_012m2VtyXxFL4RBGHzJqSN4R', 'content': '<a href="#18bacG4a">192 Oak St</a>'}]}
    
    ->

    {'role': 'assistant', 
    'content': [
        {'text': "I'll help you get information about the property at 192 Oak St.", 'type': 'text'}, 
        {'type': 'tool_use', 'name': 'get_listing', 'input': {'address': '192 Oak St'}, 'output': '<a href="#18bacG4a">192 Oak St</a>'}
        ]
    }
    
    '''
    processed_messages = []
    tool_uses_content = None
    
    for message in messages:
        if message['role'] == 'assistant' and isinstance(message['content'], list):
            # Store tool uses for next iteration
            tool_uses_content = message['content']
            continue
            
        if message['role'] == 'user' and isinstance(message['content'], list) and tool_uses_content:
            # Create mapping of tool_use_id to result content
            tool_results_map = {
                result['tool_use_id']: result['content']
                for result in message['content']
                if result['type'] == 'tool_result'
            }
            
            # Process the content list, replacing tool uses with combined use+result
            processed_content = []
            for item in tool_uses_content:
                if item.get('type') == 'tool_use':
                    processed_content.append({
                        'type': 'tool_use',
                        'name': item['name'],
                        'input': item['input'],
                        'output': tool_results_map[item['id']]
                    })
                else:
                    processed_content.append(item)
                    
            processed_messages.append({
                'role': 'assistant',
                'content': processed_content
            })
            tool_uses_content = None
            continue
            
        # Add any other messages as-is
        processed_messages.append(message)
        
    return processed_messages


def unprocess_tool_uses_and_results(messages):
    '''
    {'role': 'assistant', 
    'content': [
        {'text': "I'll help you get information about the property at 192 Oak St.", 'type': 'text'}, 
        {'type': 'tool_use', 'name': 'get_listing', 'input': {'address': '192 Oak St'}, 'output': '<a href="#18bacG4a">192 Oak St</a>'}
        ]
    }

    -> 

    {'role': 'assistant', 
    'content': [
        {'text': "I'll help you get the listing information for 192 Oak St.", 'type': 'text'}, 
        {'type': 'tool_use', 'id': 'toolu_0', 'name': 'get_listing', 'input': {'address': '192 Oak St'}}
        ]
    }

    {'role': 'user', 
    'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_0', 'content': '<a href="#18bacG4a">192 Oak St</a>'}]}
    
    '''
    
    unprocessed_messages = []
    tool_use_counter = 0
    
    for message in messages:
        if message['role'] == 'assistant' and isinstance(message['content'], list):
            assistant_message_content = []
            user_message_content = []
            for item in message['content']:
                if item.get('type') == 'tool_use':
                    # Generate unique tool use ID
                    tool_use_id = f'toolu_{tool_use_counter}'
                    tool_use_counter += 1
                    
                    # Split tool use and result into separate messages
                    tool_use = {
                        'type': 'tool_use',
                        'id': tool_use_id,
                        'name': item['name'],
                        'input': item['input']
                    }
                    assistant_message_content.append(tool_use)
                    
                    # Store tool result for later
                    user_message_content.append({
                        'type': 'tool_result',
                        'tool_use_id': tool_use_id,
                        'content': item['output'],
                    })
                else:
                    assistant_message_content.append(item)
            
            unprocessed_messages.append({
                'role': 'assistant',
                'content': assistant_message_content
            })
            unprocessed_messages.append({
                'role': 'user',
                'content': user_message_content
            })
        else:
            # Add any other messages as-is
            unprocessed_messages.append(message)
    
    return unprocessed_messages

In [8]:
messages = [
    {'role': 'user', 'content': 'Hello!'},
    {'role': 'assistant', 'content': "Hello! I'm here to help you find information about property listings. I can help you look up details about specific properties if you provide me with an address. What property would you like to learn more about?"},
    {'role': 'user', 'content': "I'm I want to put together an email for a client about the home listed at 192 Oak St. Can you pull the listing?"}
]

user_message = messages[-1]['content']
messages = messages[: -1]
messages = unprocess_tool_uses_and_results(messages)
artifacts = [] #convert_to_artifacts()



In [9]:
messages

[{'role': 'user', 'content': 'Hello!'},
 {'role': 'assistant',
  'content': "Hello! I'm here to help you find information about property listings. I can help you look up details about specific properties if you provide me with an address. What property would you like to learn more about?"}]

In [10]:
import anthropic
import re


class Conversation:
    def __init__(self, tools=None, messages=None, artifacts=None):
        self.client = anthropic.Anthropic(api_key=os.getenv('ANTHROPIC_API_KEY'))
        self.model = "claude-3-5-sonnet-20241022"
        self.messages = messages or []
        self.artifacts = artifacts or []
        self.tools = tools or []

    def say(self, message):
        system_message = self._generate_system_message(self.artifacts)
        tools = [t.schema for t in self.tools]

        self.messages.append({
            "role": "user",
            "content": message
        })

        response = self.client.messages.create(
            model=self.model,
            system=system_message,
            messages=self.messages,
            max_tokens=3000,
            temperature=0.7,
            tools=tools,
        )

        # Handle potential tool use
        while response.stop_reason == "tool_use":
            tool_result_messages = []
            for block in response.content:
                if block.type == "tool_use":
                    tool_use = block
                    tool_name = tool_use.name
                    tool_input = tool_use.input
                    tool_result = self._process_tool_call(tool_name, tool_input)
                    tool_result_messages.append({
                        "type": "tool_result",
                        "tool_use_id": tool_use.id,
                        "content": tool_result,
                    })

            self.messages.append({"role": "assistant", "content": response.content})
            self.messages.append({
                "role": "user",
                "content": tool_result_messages,
            })

            # Get final response after tool use
            response = self.client.messages.create(
                model=self.model,
                system=system_message,
                messages=self.messages,
                max_tokens=3000,
                temperature=0.7,
                tools=tools,
            )

        assistant_message = response.content[0].text
        self.messages.append({"role": "assistant", "content": assistant_message})
        artifacts, messages = self._extract_messages_and_artifacts()

        return {
            'messages': messages,
            'artifacts': artifacts,
        }

    def _process_tool_call(self, tool_name, tool_input):
        for tool in self.tools:
            if tool.name == tool_name:
                return tool.callable(**tool_input)
        raise Exception(f"Tool {tool_name} not found")

    def _generate_system_message(self, artifacts):
        artifacts_info = "\n".join([str(artifact) for artifact in artifacts])
        system_message = SYSTEM_MESSAGE

        if artifacts:
            system_message += f"""\
            <artifacts>
            {artifacts_info}
            </artifacts>
            """
        return system_message

    def _extract_messages_and_artifacts(self):
        """
        Extracts artifacts from the conversation and returns a list of messages with the artifacts replaced by anchor tags.
        """
        artifacts = self.artifacts
        new_messages = []

        # Process each message
        for message in self.messages:
            content = message['content']
            new_message = {"role": message["role"]}

            # Process string content
            if isinstance(content, str):
                new_content, message_artifacts = self._process_content(content)
                artifacts.extend(message_artifacts)
                new_message["content"] = new_content
                new_messages.append(new_message)
                continue

            # Process list content
            if isinstance(content, list):
                new_content_list = []
                for item in content:
                    # Handle string items
                    if isinstance(item, str):
                        new_item, item_artifacts = self._process_content(item)
                        artifacts.extend(item_artifacts)
                        new_content_list.append(new_item)
                        continue

                    # Handle dict-like items
                    item_dict = item.dict() if hasattr(item, 'dict') else item

                    if item_dict.get('type') == 'text':
                        new_text, text_artifacts = self._process_content(item_dict['text'])
                        artifacts.extend(text_artifacts)
                        new_item = dict(item_dict)
                        new_item['text'] = new_text
                        new_content_list.append(new_item)
                    elif item_dict.get('type') == 'tool_result':
                        new_content, content_artifacts = self._process_content(item_dict['content'])
                        artifacts.extend(content_artifacts)
                        new_item = dict(item_dict)
                        new_item['content'] = new_content
                        new_content_list.append(new_item)
                    else:
                        new_content_list.append(item_dict)

                new_message["content"] = new_content_list
                new_messages.append(new_message)

        # Remove duplicate artifacts keeping only the latest version
        seen_ids = {}
        unique_artifacts = []
        for artifact in artifacts:
            seen_ids[artifact.identifier] = artifact
        unique_artifacts = list(seen_ids.values())

        return unique_artifacts, new_messages

    def _process_content(self, text):
        """Helper method to process text content and extract artifacts"""
        artifacts = []

        # Find all artifact blocks using regex
        artifact_pattern = r'<artifact\s+identifier="([^"]+)"\s+type="([^"]+)"\s+title="([^"]+)">(.*?)</artifact>'

        # Keep track of where we last ended to build the new content
        last_end = 0
        new_content = ""

        for match in re.finditer(artifact_pattern, text, re.DOTALL):
            # Add any text before this match
            new_content += text[last_end:match.start()]

            # Extract artifact info
            identifier = match.group(1)
            type_ = match.group(2)
            title = match.group(3)
            content = match.group(4).strip()

            # Create and store artifact
            artifact = Artifact(identifier, type_, title, content)
            artifacts.append(artifact)

            # Add anchor tag
            new_content += f'<a href="#{identifier}">{title}</a>'

            last_end = match.end()

        # Add any remaining text
        new_content += text[last_end:]

        return new_content, artifacts

In [11]:
conversation = Conversation(
    tools=tools,
    messages=messages,
    artifacts=artifacts,
)

response = conversation.say(user_message)

C:\Users\a84189971\AppData\Local\Temp\ipykernel_6900\564943585.py:121: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  item_dict = item.dict() if hasattr(item, 'dict') else item


In [12]:
print(response)

{'messages': [{'role': 'user', 'content': 'Hello!'}, {'role': 'assistant', 'content': "Hello! I'm here to help you find information about property listings. I can help you look up details about specific properties if you provide me with an address. What property would you like to learn more about?"}, {'role': 'user', 'content': "I'm I want to put together an email for a client about the home listed at 192 Oak St. Can you pull the listing?"}, {'role': 'assistant', 'content': [{'text': "I'll help you get the listing information for 192 Oak St.", 'type': 'text'}, {'id': 'toolu_014qPwweB4GpRnDEdxgVywEd', 'input': {'address': '192 Oak St'}, 'name': 'get_listing', 'type': 'tool_use'}]}, {'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_014qPwweB4GpRnDEdxgVywEd', 'content': '<a href="#18bacG4a">192 Oak St</a>'}]}, {'role': 'assistant', 'content': 'I\'ve retrieved the listing details for 192 Oak St. It\'s a 3-bed, 2-bath ranch-style home in Cedar Rapids, Iowa, listed a

In [13]:
messages = response['messages']
artifacts = response['artifacts']

In [14]:
messages

[{'role': 'user', 'content': 'Hello!'},
 {'role': 'assistant',
  'content': "Hello! I'm here to help you find information about property listings. I can help you look up details about specific properties if you provide me with an address. What property would you like to learn more about?"},
 {'role': 'user',
  'content': "I'm I want to put together an email for a client about the home listed at 192 Oak St. Can you pull the listing?"},
 {'role': 'assistant',
  'content': [{'text': "I'll help you get the listing information for 192 Oak St.",
    'type': 'text'},
   {'id': 'toolu_014qPwweB4GpRnDEdxgVywEd',
    'input': {'address': '192 Oak St'},
    'name': 'get_listing',
    'type': 'tool_use'}]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_014qPwweB4GpRnDEdxgVywEd',
    'content': '<a href="#18bacG4a">192 Oak St</a>'}]},
 {'role': 'assistant',
  'content': 'I\'ve retrieved the listing details for 192 Oak St. It\'s a 3-bed, 2-bath ranch-style home in

In [15]:
artifacts

[Artifact(identifier="18bacG4a", title="192 Oak St")]

In [16]:
len(messages)

6

In [17]:
for m in messages:
    print(m)
    print()

{'role': 'user', 'content': 'Hello!'}

{'role': 'assistant', 'content': "Hello! I'm here to help you find information about property listings. I can help you look up details about specific properties if you provide me with an address. What property would you like to learn more about?"}

{'role': 'user', 'content': "I'm I want to put together an email for a client about the home listed at 192 Oak St. Can you pull the listing?"}

{'role': 'assistant', 'content': [{'text': "I'll help you get the listing information for 192 Oak St.", 'type': 'text'}, {'id': 'toolu_014qPwweB4GpRnDEdxgVywEd', 'input': {'address': '192 Oak St'}, 'name': 'get_listing', 'type': 'tool_use'}]}

{'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_014qPwweB4GpRnDEdxgVywEd', 'content': '<a href="#18bacG4a">192 Oak St</a>'}]}

{'role': 'assistant', 'content': 'I\'ve retrieved the listing details for 192 Oak St. It\'s a 3-bed, 2-bath ranch-style home in Cedar Rapids, Iowa, listed at $185,000. Th

In [18]:
len(process_tool_uses_and_results(messages))

5

In [19]:
for m in process_tool_uses_and_results(messages):
    print(m)
    print()

{'role': 'user', 'content': 'Hello!'}

{'role': 'assistant', 'content': "Hello! I'm here to help you find information about property listings. I can help you look up details about specific properties if you provide me with an address. What property would you like to learn more about?"}

{'role': 'user', 'content': "I'm I want to put together an email for a client about the home listed at 192 Oak St. Can you pull the listing?"}

{'role': 'assistant', 'content': [{'text': "I'll help you get the listing information for 192 Oak St.", 'type': 'text'}, {'type': 'tool_use', 'name': 'get_listing', 'input': {'address': '192 Oak St'}, 'output': '<a href="#18bacG4a">192 Oak St</a>'}]}

{'role': 'assistant', 'content': 'I\'ve retrieved the listing details for 192 Oak St. It\'s a 3-bed, 2-bath ranch-style home in Cedar Rapids, Iowa, listed at $185,000. The property has 1,450 square feet of living space on a quarter-acre lot. You can find the complete listing details including features and descripti

In [20]:
[artifact.dict() for artifact in artifacts]

[{'identifier': '18bacG4a',
  'type': 'application/json',
  'title': '192 Oak St',
  'content': '{\n    "address": "192 Oak St",\n    "city": "Cedar Rapids",\n    "state": "IA",\n    "zip": "52402",\n    "price": 185000,\n    "beds": 3,\n    "baths": 2,\n    "sqft": 1450,\n    "lot_size": 0.25,\n    "year_built": 1978,\n    "description": "Charming ranch-style home in established neighborhood. Updated kitchen with new appliances. Finished basement, attached 2-car garage, fenced backyard with mature trees. Close to schools and shopping.",\n    "features": [           \n        "Central air",\n        "Forced air heating",\n        "Hardwood floors",\n        "Updated kitchen",\n        "Finished basement",\n        "Attached garage",\n        "Fenced yard"\n    ],\n}'}]

In [21]:
artifacts[0]

Artifact(identifier="18bacG4a", title="192 Oak St")

In [22]:
artifacts[0].dict()

{'identifier': '18bacG4a',
 'type': 'application/json',
 'title': '192 Oak St',
 'content': '{\n    "address": "192 Oak St",\n    "city": "Cedar Rapids",\n    "state": "IA",\n    "zip": "52402",\n    "price": 185000,\n    "beds": 3,\n    "baths": 2,\n    "sqft": 1450,\n    "lot_size": 0.25,\n    "year_built": 1978,\n    "description": "Charming ranch-style home in established neighborhood. Updated kitchen with new appliances. Finished basement, attached 2-car garage, fenced backyard with mature trees. Close to schools and shopping.",\n    "features": [           \n        "Central air",\n        "Forced air heating",\n        "Hardwood floors",\n        "Updated kitchen",\n        "Finished basement",\n        "Attached garage",\n        "Fenced yard"\n    ],\n}'}

In [23]:
for m in unprocess_tool_uses_and_results(process_tool_uses_and_results(messages)):
    print(m)
    print()

{'role': 'user', 'content': 'Hello!'}

{'role': 'assistant', 'content': "Hello! I'm here to help you find information about property listings. I can help you look up details about specific properties if you provide me with an address. What property would you like to learn more about?"}

{'role': 'user', 'content': "I'm I want to put together an email for a client about the home listed at 192 Oak St. Can you pull the listing?"}

{'role': 'assistant', 'content': [{'text': "I'll help you get the listing information for 192 Oak St.", 'type': 'text'}, {'type': 'tool_use', 'id': 'toolu_0', 'name': 'get_listing', 'input': {'address': '192 Oak St'}}]}

{'role': 'user', 'content': [{'type': 'tool_result', 'tool_use_id': 'toolu_0', 'content': '<a href="#18bacG4a">192 Oak St</a>'}]}

{'role': 'assistant', 'content': 'I\'ve retrieved the listing details for 192 Oak St. It\'s a 3-bed, 2-bath ranch-style home in Cedar Rapids, Iowa, listed at $185,000. The property has 1,450 square feet of living spa